## P0 Compiler Optimization: Draft Report
#### COMP SCI 4TB3/6TB3, McMaster University
#### Original Author: Emil Sekerinski, February 2017
---
#### Optimization Authors: Sherwin Chang (001426383), Arian Sohrabi (001409334)
#### Date: April 2017
---

## Optimization Draft Report

When working with computers, it is easy to have inefficient codes likely due to ease of understanding. As such, when program size grows, the runtime speed will suffer and results in a slower program. The goal of the 'P0 Compiler Optimization' is to implement various optimization techiniques on the code generated by the P0 compiler. At the current state, the P0 compiler already have implemented some optimization techniques (i.e. constant folding and constant propagation), the plan is to expand on these techniques. Doing this will in turn improve the speed at runtime, but will be done at the cost of overall code length and/or "cleanliness".

##### Assumptions
The optimization makes assumption that the optimization techniques will be applied for the MIPS code generated by the compiler, and not for the compiler itself.


### Optimization Techniques
The below will list the various optimization techniques that is in some degree implemented. Each of the optimization have a varying degree of complexity and thus will be discussed inside the block of that optimization.

Note: Each optimization will have an examples done in python to help explain how the technique works.

##### Organization
The optimizations of the report will be organized in this order.

Description >> Python code demonstration >> MIPS code before implementation >> Explaination of implementation >> Testing of Implementation >> Nuances/Side notes.

##### Complexity
The complexity of each optimization will be roughly defined as the following. (Note: the length is not the only consideration, but also difficulty of implementation, but that varies amongst optimization, and will be discussed on a case-to-case basis.)

- Very Easy: The optimization required 1 or 2 lines of code to implement.

- Easy: The optimization required 3 to 8 lines of code to implement. 

- Medium: The optimization required 9 to 20 lines of code to implement.

- Hard: The optimization required 20 or more lines of change.

- Very Hard: The optimization required a restructure of the p0 language.

In [2]:
import nbimporter
from P0 import compileString

#### Integer Multiply/Divide Optimization
Integer multiplication and division can be replaced with a shift operation if one of the operands is a power of two. Shift operations are faster than multiply on mips architectue, therefor this will reduce the running time of the executable. The amount shifted will be based on the log base 2 of the constant number.

In [3]:
def before (n): #Multiplication
    return n * 2

def after (n):
    return n << 1

In [4]:
def before (n): #Division
    return n / 2

def after (n):
    return n >> 1

The following shows the MIPS Code before optimization of the example provided below.
```
	.data
z_: .space 4
y_: .space 4
x_: .space 4
n_: .space 4
	.text
	.globl main
	.ent main
main:	
    addi $t2, $0, 5
    sw $t2, n_
    lw $t5, n_
    mul $t5, $t5, 4
    sw $t5, x_
    addi $t3, $0, 8
    lw $t6, n_
    mul $t3, $t3, $t6
    sw $t3, y_
    lw $t1, x_
    div $t1, $t1, 4
    sw $t1, z_
    li $v0, 10
    syscall
	.end main
```

In order to implement the Integer Multiply/Divide Optimization, changes are needed to be made in genBinaryOp() in CGmips. Where when it is a multiplcation, and either of the value is a constant, it checks if the val in the constant is of a power of 2. If it is, use 'srl' instead of 'mul'. The same idea applies for division but only if the right value is a power of 2.

The code below should now generate an optimized MIPS code. Which replaces mul and div with srl and sll respectively.

In [28]:
compileString("""
program p;
  var n, x, y, z: integer;
  begin n := 5;
    x := n * 4;
    y := 8 * n;
    z := x div 4
  end
""")

	.data
z_:	.space 4
y_:	.space 4
x_:	.space 4
n_:	.space 4
	.text
	.globl main
	.ent main
main:	
	addi $t4, $0, 5
	sw $t4, n_
	lw $t1, n_
	srl $t1, $t1, 2
	sw $t1, x_
	lw $t7, n_
	srl $t7, $t7, 3
	sw $t7, y_
	lw $t6, x_
	sll $t6, $t6, 2
	sw $t6, z_
	li $v0, 10
	syscall
	.end main


This optimization will only work for constant values. We tried to implement it for x:= x mul y, when y := 4, but since Var type doesn't keep a value, we can't check if y is a power of two.

For this optimization, Arian worked on implementation and examples, and Sherwin worked on explanation.

Complexity: Easy. 
Because the ground work is done in the following optimization, the only changes needed to be made are to check when the constant value is a power of two, calculate the logarithmic value to be shifted, and put srl or sll accordingly. 

#### Arithmetic Code Optimization

In arithmetics calculations, some constant calculations can be reduced to exclude redundant calculations. Such as with adding and subtracting by 0, multipling and dividing by 1, and multipling by 0.  (Most of this is done for Lab 11)

In [6]:
def before (n): #Arithmetics
    x1 = 1 + 0
    x2 = 0 + x
    x3 = x - 0
    x4 = x * 1
    x5 = 1 * x
    x6 = x / 1
    x7 = x * 0
    x8 = 0 / y
    x9 = y / y
    x10 = y - y
    
def after (n):
    x1 = 1
    x2 = x
    x3 = x
    x4 = x
    x5 = x
    x6 = x
    x7 = 0
    x8 = 0
    x9 = 1
    x10 = 0

The example provided would have generated the following MIPS code.
```
	.data
z_: .space 4
y_: .space 4
x_: .space 4
	.text
	.globl main
	.ent main
main:	
	addi $t7, $0, 5
	sw $t7, x_
    lw $t6, x_
	add $t6, $t6, 0
	sw $t6, y_
    lw $t2, x_
	add $t1, $0, $t2
    sw $t1, z_
	lw $t3, z_
    sub $t3, $t3, 0
    sw $t3, x_
	lw $t8, z_
    mul $t8, $t8, 1
    sw $t8, y_
	addi $t0, $0, 1
	lw $t4, x_
    mul $t0, $t0, $t4
	sw $t0, z_
    lw $t5, x_
	add $t5, $t5, 3
	sw $t5, x_
    addi $t7, $0, 3
    lw $t6, y_
	add $t7, $t7, $t6
    sw $t7, y_
    lw $t1, x_
    div $t1, $t1, 1
    sw $t1, z_
	lw $t2, x_
    mul $t2, $t2, 0
    sw $t2, x_
	lw $t8, x_
    mul $t3, $0, $t8
	sw $t3, y_
    lw $t4, y_
	div $t0, $0, $t4
    sw $t0, z_
	lw $t5, x_
    lw $t7, x_
    div $t5, $t5, $t7
    sw $t5, x_
    lw $t6, x_
    lw $t1, y_
    div $t6, $t6, $t1
    sw $t6, y_
    lw $t2, z_
    lw $t3, z_
    sub $t2, $t2, $t3
    sw $t2, z_
	li $v0, 10
	syscall
	.end main
```

To allow for simplification of the basic arithmetics, changes are only needed to be made in CGmips. In particularly, to put and to genBinaryOp. The changes to put basically allows the reverse x + 3 and 3 + y to both generate the same instructions. The changes to genBinaryOp are to check for the conditions for each of the cases where they can be simplified. For example, when x:= y + 0, it will have checked that 0 is of type Const and adding a 0 will just result in x := y, as such it will return y.

In addition to the work done for lab 11, the optimization also generates the same behavour for when multiplying by 0 and when the dividend is 0. The optimization also does the case where both the dividend and divisor is the same Var. This is done in term() in the p0 class before getting the factor() with SC.val. This is done in so that the parser can check if the left and right of the division operation is the same variable, as the genBinaryOp doesn't actually take in the variable symbol used. The same idea applies to implementing subtraction of the same variable (x:=y-y).

The below function is to test the implementation. (New additions are near the bottom)

In [27]:
compileString("""
program p;
  var x, y, z: integer;
  begin x := 5;
    y := x + 0;
    z := 0 + x;
    x := z - 0;
    y := z * 1;
    z := 1 * x;
    x := x + 3;
    y := 3 + y;
    z := x div 1;
    x := x * 0;
    y := 0 * x;
    z := 0 div y;
    x := x div x;
    y := x div y;
    z := z - z
  end
""")

	.data
z_:	.space 4
y_:	.space 4
x_:	.space 4
	.text
	.globl main
	addi $t4, $0, 5
	sw $t4, x_
	lw $t1, x_
	sw $t1, y_
	lw $t7, x_
	sw $t7, z_
	lw $t6, z_
	sw $t6, x_
	lw $t8, z_
	sw $t8, y_
	lw $t5, x_
	sw $t5, z_
	lw $t2, x_
	add $t2, $t2, 3
	sw $t2, x_
	lw $t0, y_
	add $t0, $t0, 3
	sw $t0, y_
	lw $t3, x_
	sw $t3, z_
	sw $0, x_
	sw $0, y_
	sw $0, z_
	addi $t4, $0, 1
	sw $t4, x_
	lw $t1, x_
	lw $t7, y_
	div $t1, $t1, $t7
	sw $t1, y_
	sw $0, z_
	li $v0, 10
	syscall
	.end main


Like the optimization before this, this optimization will only work for constant values. We tried to implement it for x:= x mul y, when y := 0, but this is not possible as the values of registers and variables are not stored by the compiler. Also, keep in mind that the compiler does not check for when x := x div 0, this has more to do with error checking so we opted not to deal with this case as it is not really an optimization. Likewise, testing for x := Const - x also returns an erronous output, and that has more to do with the parser for putOp in CGmips.

For this optimization, Sherwin and Arian both worked on implementation (different cases for each person), and Sherwin worked on examples and explanation.

Complexity: Medium. 
While the new changes are not particularly difficult, the groundworks was done for lab 11, as such the difficulty is at about a medium to include the work done for lab 11. And this is because it is important to consider all the possible cases, and figuring out how to edit the generator so that it properly generates a different instruction. Doing the x := y div y is a bit trickier as it is done in term() and not genBinaryOp, and having to find a way to equate the variables as the parser doesn't keep the variable letter in the ST class also created some complications.

#### Dead Code Elimination
Some code comes off as being redundant, unused, or unreachable. As such, they can be elimited without having an effect on the program. The three types that we try to optimize here are;

1. Elimination of uninitialized variables and constants. 
2. Removing unused initializations.
3. Combining redundant initializations.

In [26]:
g = 0

def before():
    n # case 1 :uninitialized variable
    k = 1 
    k = k - 1 #case 3: redundant initializations
    g = 2 #case 2: unused initializations
    g = 3
    g = 3 #redundant statement
    return k + g
    g = 4 #case 4: unreachable statement

def after():
    k = 1 - 1
    g = 3
    return k + g

The following the the MIPS code generated by the test case before optimization.
```
	.data
z_: .space 4
y_: .space 4
x_: .space 4
	.text
	.globl main
	.ent main
main:	
	addi $t7, $0, 1
	sw $t7, x_
    addi $t6, $0, 3
    sw $t6, y_
	addi $t1, $0, 5
	sw $t1, y_
    addi $t2, $0, 7
    sw $t2, x_
	lw $t3, x_
    add $t3, $t3, 9
    sw $t3, x_
L1:	
	beq $0, $0, L2
L3:	
	addi $t8, $0, 11
	sw $t8, x_
    addi $t0, $0, 13
    sw $t0, y_
	b, L1
L2:	
L4:	
	addi $t4, $0, 1
	beq $t4, $0, L5
L6:	
	addi $t5, $0, 15
	sw $t5, x_
    addi $t7, $0, 17
    sw $t7, y_
	b, L4
L5:	
	addi $t6, $0, 19
	sw $t6, x_
    li $v0, 10
	syscall
	.end main
```

The implementation of this optimization is done based on the case.

For Case 1:

- The optimization for case 1 looks for any declared variables that are not used at all by the parser and remove them. This is done with the help of the following variables:
- A global list of all declared variables is kept in p0 called declared_identifiers[]. 
- A global list of all initialized variables is kept in p0 called initialized_identifiers[]. 
- A global variable called number_of_declarations is kept in p0 that holds the corresponding number.
- A global variable called unused_declarations is computed using declared_identifiers-initialized_identifiers. This computation takes place in check_dead_code() which is called at the end of program() in p0.
- check_dead_code() calls remove_uninitialized_vars() in CGmips.
- remove_uninitialized_vars() iterates through asm[1:number_of_declarations] and checks if asm[i][0] (corresponding to the declared variable) exists in unused_declarations. If so it removes that index from asm.
- Values are reset with renew_global_vars() with every compileString() call


For Case 2:

- The optimization for case 2 removes consequtive declarations. This is done as follows:
- A global variable consecutive_initializations is set to false.
- When initializing a variable, look for the value of consecutive_initializations.
- if consecutive_initializations != [current_variable, True] then create the assembly code for initialization of the variable. And then set consecutive_initializations=[current_variable, True]
- else : remove the assembly code for the previous unused initialization using asm.pop(). Then create the assembly code for the new initialization of the variable. set consecutive_initializations=[current_variable, True].

For Case 3 (AKA Instruction Combining):

- Not Implemented.
- The implementation would have to require to build on case 2 and have to keep a stack of the variable and combine all of them that are not used by another variable. This will probably take too much time to do.
- A more detailed implementation guide listed further down below.

For Case 4:

- Parser doesn't really return anything, or break out of anything. So only case of unreachable code is in an infinite loop.
- In statement(), new conditions are made to check for an infinite loop. (When it is a constant boolean with val == 1)
- When an inifite loop occurs, the parser will skip past everything after the loop as they can't be reached.
- If the loop condition is a constant false, it will just skip past the loop. (See next optimization)

The following example will test all of the cases.

What to look for:

- Removal of the Var z
- Deletion of y := 3
- Combining of x := 7 + 9
- Skipping of the entire while false loop (x := 11, and y := 13)
- Deletion of x := 19, because it is after the infinite loop

In [17]:
compileString("""
program p;
  var x, y, z: integer;
  begin x := 1;
    y := 3;
    y := 5;
    x := 7;
    x := x + 9;
    while false do
      begin
        x := 11;
        y := 13
      end;
    while true do
      begin
        x := 15;
        y := 17
      end;
    x := 19
  end
""")

	.data
y_:	.space 4
x_:	.space 4
	.text
	.globl main
	.ent main
main:	
	addi $t4, $0, 1
	sw $t4, x_
	addi $t7, $0, 5
	sw $t7, y_
	lw $t8, x_
	add $t8, $t8, 9
	sw $t8, x_
L1:	
	addi $t5, $0, 1
	beq $t5, $0, L2
L3:	
	addi $t2, $0, 15
	sw $t2, x_
	addi $t0, $0, 17
	sw $t0, y_
	b, L1
L2:	
	li $v0, 10
	syscall
	.end main


Below is the implemenation concept by Arian for Case 3:

Conditions:

1. There has to be two initialization statements of the same variable.
2. There has to be no use of the variable between the first and second initializations.
3. The second initialization must be of the form Ident:=Ident + Constant


Optimization:

1. The first initialization is of the form Ident := expression
2. The second is of the form Ident:=Ident + Constant
3. Combine the two to get Ident := expression + Constant


Adjustments for implementation:

- Condition two is simplified to only account for consecutive initiallizations
- Otherwise we have to keep track of the position of each initialization and the corresponding assembly code and this will have a large complexity.


Implementation ideas:

- Keep a list of identifiers used in the last evaluated expression. call this vars[]

Case 3: PSEUDO CODE
```
if statement == initialization:
	first_variable = SC.val
		first_val = expression()
	if next_statement == initialization:
		second_variable = SC.val
		second_value = expression()
		if first_variable == second_variable and type(value) == Const and first_variable in vars[]:
			initialize (first_variable := first_val + second_value)
		else:
			initialize (first_variable := first_val)		
```

Case 1:

- At first we planned to remove the declaration and initiallization of all variables that were not used in an expression. But this proved difficult. Removing one variable could have caused another variable to become useless. so we stuck to removing uninitialized variables.
- Complexity: Medium. Because the difficulty comes from recognizing that asm contains the lines with the variable declaration, and just need to keep track and remove from asm the variables that are not used.

Case 2:

- At first we planned to remove the unused initialization of a variable through out the code. But this proved difficult because we have to keep track of the position of every initialization in the assembly code. Then we have to keep track of the length of the assembly code so that we can remove unused instructions. And because the parser rarely generates an output, implementing it will create problems with other tests.
- And also, because instruction combining (Case 3) is not implemented, the parser will simply remove the previous statement regardless of it being used by the current or not. This is a problem/error that would be fixed with implementation of case 3.
- Complexity: Easy. This case requires keeping track for statements in sequence. And remove the ones that comes before from asm.


Case 3:

- Not implemented.
- Common Subexpression Optimization can be built on this if Case 3 was implemented. Where in some expressions, a duplicate sequence can occur (they have common subexpression). The duplication can be recognized (stored) and just calculate the sequence one time. But that is very difficult to do as the parser doesn't keep track of the expressions being parsed.
- Complexity: Medium/Hard. The impletation would have required keeping track of what variables are being used in the expression, this is something not done by the parser, and thus would have required additional variables to do.

Case 4:

- The tricky part about this case is to also consider the begin and end block, and parse that block correctly. Which means regonizing where to skip or else an error might occur. 
- Complexity: Easy-Medium. As seen in the following optimization, most of the work is already completed. The same concept just has to be carried over to this optimization where it skips the while false loop and skip the statements after a while true loop.

For the optimization, Arian worked on everything Case 1 and Case 2 including examples and report, Sherwin worked on everything Case 4. Case 3 description was made by the both of us.

#### If-Statement Elimination
After compiliation, it can be determined if some of the if-statements takes a constant boolean as the expression (with help of constant folding). When that is the case it is no longer necessary to generate an if-statment. (Most of this is done for Lab 11)

In [10]:
def before():
    g = 0
    k = 1
    if True:
        if g > k:
            return 0
        if False: 
            return 1
        else:
            return 2
    else:
        return 3
    
    
def after():
    return 2

The first example provided is a modified version of the Lab Question 2. And the MIPS code shown below is before the optimization.
```
    .data
x_: .space 4
	.text
	.globl main
	.ent main
main:	
    addi $t7, $0, 1
    addi $t8, $0, 2
    ble $t7, $t8, L1
L2:	
    addi $t2, $0, 1
    beq $t2, $0, L3
L4:
    addi $t6, $0, 3
    sw $t6, x_
    b, L5
L3:	
    addi $t5, $0, 5
    sw $t5, x_
L5:	
	b, L6
L1:	
    beq $0, $0, L7
L8:	
    addi $t0, $0, 7
    sw $t0, x_
    b, L9
L7:	
    addi $t3, $0, 9
    sw $t3, x_
L9:	
L6:	
	bne $0, $0, L10
L11:	
	addi $t1, $0, 1
	beq $t1, $0, L12
L13:	
    addi $t4, $0, 11
    sw $t4, x_
    b, L14
L12:	
    addi $t7, $0, 13
    sw $t7, x_
L14:	
    b, L15
L10:	
    beq $0, $0, L16
L17:	
    addi $t8, $0, 15
    sw $t8, x_
    b, L18
L16:	
    addi $t2, $0, 17
    sw $t2, x_
L18:	
L15:	
    li $v0, 10
    syscall
    .end main
```

The second test shows the addition to lab 11 which also allows non-constant boolean expressions into the nested if statments. The MIPS code before any optimization is shown below.

```
    .data
x_: .space 4
    .text
    .globl main
    .ent main
main:	
    lw $t5, x_
    addi $t1, $0, 2
    ble $t5, $t1, L1
L2:	
    lw $t6, x_
    addi $t2, $0, 4
    bne $t6, $t2, L3
L4:	
    addi $t0, $0, 1
    beq $t0, $0, L5
L6:	
    addi $t8, $0, 3
    sw $t8, x_
    b, L7
L5:	
    addi $t4, $0, 5
    sw $t4, x_
L7:	
    b, L8
L3:	
    addi $t7, $0, 7
    sw $t7, x_
L8:	
    b, L9
L1:	
    beq $0, $0, L10
L11:	
    addi $t3, $0, 9
    sw $t3, x_
    b, L12
L10:	
    addi $t5, $0, 11
    sw $t5, x_
L12:	
L9:	
    li $v0, 10
    syscall
    .end main
```



For this optimization, changes only needed to be made in the statement() function in p0. Which basically checks for a symbol and calls functions base on what the statement does. For this optimization, changes only needed to be made when SC.sym == IF. There are multiple changes needed to be made. First, it checks if the expression which follows the 'if' is a constant expression (using constant folding); if it is and it comes out True, then it only recursively calls statement() on the 'then' part of the expression and skip the 'else', but if it comes out False, it will skip past the expression in the 'then' part and only call the expression after the else. 

To allow this for nested if statements, it first checks if there is a 'then' after get the x := expression(). If there is, then it also checks for a 'if' after the 'then', if that 'if' exists, do that if statment by recursively calling statement(), assuming that x is a const and is True. If x is False, it skips the entire nested if statements by convolutedly count the ifs and elses and skip base on that number.

A similar concept applies to the while loop optimization (Case 4) above.

Additionally, the functionality has been expanded from lab 11 to also allow boolean expressions (i.e.if x > 2) for the nested loops. And this is done by checking to see if the expression in the parent of the nested loop (if expr then if...) is a variable boolean, if it is, then generate the labels that is needed.

Minor changes also needed to be made in expression() to allow constant folding on relational operators.

The below shows two tests. The first is a modified version of Lab 11's test, which test for all of the functionality in done in lab 11, this is also the example used by the MIPS code above. The second test shows that the optimization now also allows for non-constant boolean expressions in the conditions the parent then-if statement. When comparing the MIPS code shown above, and the MIPS that is generated after optimization, it shows that the length and speed is greatly reduced as it removes unnessesary constant if statements.

Notice that for the second test, the results omitted x:=5 and x:=9 as those can't be reached.

In [21]:
compileString("""
program p;
  var x: integer;
  begin
    if 1 > 2 then
      if true then x := 3 else x := 5
    else
      if false then x := 7 else x := 9;
    if 0 = 0 then
      if true then x := 11 else x := 13
    else
      if false then x := 15 else x := 17
  end
""")

	.data
x_:	.space 4
	.text
	.globl main
	.ent main
main:	
	addi $t4, $0, 9
	sw $t4, x_
	addi $t1, $0, 11
	sw $t1, x_
	li $v0, 10
	syscall
	.end main


In [22]:
compileString("""
program p;
  var x: integer;
  begin
    if x > 2 then
      if x = 4 then
        if true then x := 3 else x := 5
      else
        x := 7
    else
      if false then x := 9 else x := 11
  end
""")

	.data
x_:	.space 4
	.text
	.globl main
	.ent main
main:	
	lw $t4, x_
	addi $t1, $0, 2
	ble $t4, $t1, L1
L2:	
	lw $t7, x_
	addi $t6, $0, 4
	bne $t7, $t6, L3
L4:	
	addi $t8, $0, 3
	sw $t8, x_
	addi $t5, $0, 7
	sw $t5, x_
L5:	
	b, L6
L1:	
	addi $t2, $0, 11
	sw $t2, x_
L6:	
	li $v0, 10
	syscall
	.end main


The implementation for this is fairly complicated. And has ran into multiple problems. For one, checking for constant and skipping them is not that intuitive. It is neccessary to call statement() on the only the expressionthat is being called, and skipping (using getSym()) on the symbols that we do not care about. I.e. if true then x else y, we would have to ensure skipping on the else y part of the code.

Furthermore, skipping for nested if-statments are also not very intuitive. For this implementation it counts the amount of ifs and elses and skip that amount to get to the next statement, and this took a while to figure out.

In terms of problems with this implementation. After trying couple tests, we determined a problem where suppose we want:
```
      if x = 4 then
        if true then x := 3
      else
        x := 5
```
This is not possible. Because of the implementation of if and else statements, we can't tell where a block starts and where it ends. and the parser will simply assume that the else belongs to the nested if-statement and not the parent. And the parent will simply not have an else statement.

For this optimization, Sherwin and Arian both worked on the implementation, the check for boolean constants and skipping the false ones are mostly done by Arian, Sherwin made the nested if-statements to work and expanded on what was done previously to also allow Var type booleans. Also, Sherwin worked on examples and explanation.

Complexity: Hard.
Keep in mind that the complexity also consider the work done for lab 11. Including that work, the complexity is hard because it requires a good understanding of how the language works and how statement() works in particular. While the implementation is fairly round-about, and probably could be simplified, it ended up being fairly long (because it needed to skip particular parts). Also because there are many errors encountered along the way.

#### Loop-Invariant Code Motion
When a loop has loop-invariant codes--codes which have the same value before and after the loop--they can be move outside of the loop. So that they do not have to ran with every iteration of the loop.

In [25]:
def before (i):
    while True:
        a = 5
        b = a + a
        c = i / 2
        if i == 10:
            return a + b + c
        i += 1
        
def after (i):
    a = 5 #a and b never changes
    b = a + a
    while True:
        c = i / 2
        if i == 10:
            return a + b + c
        i += 1

This optimization is not implemented. This is because it is very difficult (near impossible) to do.

Originally, we thought to only do this for the case where in a while loop, there is only one expression, and that expression is a constant.

```
while x < 2 do
    x := 3
```

And remove the while loop. This is pretty much impossible without making significant changes to the parser. This is because in order to generate a while loop, the parser will call CG.genTarget() and CG.genCond(x), to create the labels and loop conditions, and then it will follow with getting the statement(). There is no way to look ahead into the parser and determine if the expression() is a Const without it incrementing the symbols and cause problems with statement(). Which means a lot of changes needed to be made to the parser.

The second option is to remove the lines of MIPS code from asm after the parsing. But there are a lot of possible patterns that need to be matched and thus would be near impossible and very time consuming.

The below is the test originally planned for this optimization.

In [24]:
 compileString("""
program p;
  var x, y: integer;
  begin
    x := 0;
    while x < 10 do
        y := 1;
    x := x + y
  end
""")

	.data
y_:	.space 4
x_:	.space 4
	.text
	.globl main
	sw $0, x_
	lw $t4, x_
	addi $t1, $0, 10
L3:	
	bge $t4, $t1, L1
L2:	
	addi $t7, $0, 1
	sw $t7, y_
	b, L3
L1:	
	lw $t6, x_
	lw $t8, y_
	add $t6, $t6, $t8
	sw $t6, x_
	li $v0, 10
	syscall
	.end main


### Referenes
All optimization references the following two sites. Which explain the general concept of the optimization. We based the implementation on the ideas presented, and our knowledge of the P0 compiler.

http://www.compileroptimizations.com

https://en.wikipedia.org/wiki/Optimizing_compiler